# Price Data

We try to download available price data for all assets in our universe, as defined in the 'names_and_tickers' folder. The APIs return the following data: ``date``, ``open``, ``high``, ``low``, ``close``, ``adj_close``, ``volume`` (column names may differ between APIs). We only keep the ``adj_close`` column for each ticker. 

We join all data for one asset type into a single dataframe and save it to ``data/asset_data/raw/prices`` as csv. Then further processing (computing returns, missing data statistics etc.) is done and the results are saved to ``data/asset_data/prices`` and ``data/asset_data/returns``.

### Sources

- Stocks, ETFs and Cryptocurrencies: eodhd
- Commodities (futures only): yahoo finance

In [1]:
import pandas as pd
import asset_data_utils as adu

# timeframe of requested data
start_date = "2016-01-01"
end_date = "2024-06-04" # we need data AT LEAST to the end of 2023 (to have +1 year of performance data after the video scraping cutoff date). 

## Downloads

### Stocks

In [ ]:
tickers_path = "names_and_tickers/eodhd_stocks.csv"
tickers_colname = "Code" # for cryptos: "Code_clean"
save_path = "raw/prices/stocks_adj_close.csv"
progress_path = "raw/prices/temp_stocks_progress.csv"
exchange_symbol ="US" # stocks/etfs
min_time_between_requests = 0.1 # 0.06 sec is the eodhd minute limit (1k/min)
######################################################################################################################################################
# load tickers
tickers = pd.read_csv(tickers_path, sep=";")[tickers_colname].tolist()
print(f"Loaded {len(tickers)} tickers.")

# download
exceptions_list = adu.full_download(tickers=tickers, 
                                exchange_symbol=exchange_symbol, 
                                start_date=start_date, 
                                end_date=end_date, 
                                save_path=save_path, 
                                progress_path=progress_path, 
                                min_time_between_requests=min_time_between_requests,
                                return_exceptions_list=True)


### ETFs

In [ ]:
tickers_path = "names_and_tickers/eodhd_etfs.csv"
tickers_colname = "Code" # for cryptos: "Code_clean"
save_path = "raw/prices/etfs_adj_close.csv"
progress_path = "raw/prices/temp_etfs_progress.csv"
exchange_symbol ="US" # stocks/etfs
min_time_between_requests = 0.1 # 0.06 sec is the eodhd minute limit (1k/min)
######################################################################################################################################################
# load tickers
tickers = pd.read_csv(tickers_path, sep=";")[tickers_colname].tolist()
print(f"Loaded {len(tickers)} tickers.")


# download
exceptions_list = adu.full_download(tickers=tickers, 
                                exchange_symbol=exchange_symbol, 
                                start_date=start_date, 
                                end_date=end_date, 
                                save_path=save_path, 
                                progress_path=progress_path, 
                                min_time_between_requests=min_time_between_requests,
                                return_exceptions_list=True)

### Crypto

In [ ]:
tickers_path = "names_and_tickers/eodhd_cryptos.csv"
tickers_colname = "Code_clean" # for cryptos: "Code_clean"
save_path = "raw/prices/cryptos_adj_close.csv"
progress_path = "raw/prices/temp_cryptos_progress.csv"
exchange_symbol ="CC" # stocks/etfs
min_time_between_requests = 0.1 # 0.06 sec is the eodhd minute limit (1k/min)
######################################################################################################################################################
# load tickers
tickers = pd.read_csv(tickers_path, sep=";")[tickers_colname].tolist()
print(f"Loaded {len(tickers)} tickers.")


# download
exceptions_list = adu.full_download(tickers=tickers, 
                                exchange_symbol=exchange_symbol, 
                                start_date=start_date, 
                                end_date=end_date, 
                                save_path=save_path, 
                                progress_path=progress_path, 
                                min_time_between_requests=min_time_between_requests,
                                return_exceptions_list=True)

### Commodities

For commodities, we have to combine data from eodhd (etfs/stocks) and yahoo finance (futures). 

In [77]:
tickers_path = "names_and_tickers/yahoo_eodhd_commodities.csv"
tickers_colname = "Code" # for cryptos: "Code_clean"
data_source_colname = "pricing_source"
save_path = "raw/prices/commodities_adj_close.csv"
progress_path = "raw/prices/temp_commodities_progress.csv"
exchange_symbol ="US" # stocks/etfs
min_time_between_requests = 0.1 # 0.06 sec is the eodhd minute limit (1k/min)
######################################################################################################################################################
# load tickers and data sources
df = pd.read_csv(tickers_path, sep=";")
tickers = df[tickers_colname].tolist()
data_source_list = df[data_source_colname].tolist()
print(f"Loaded {len(tickers)} tickers.")


# download
exceptions_list = adu.full_download(tickers=tickers, 
                                exchange_symbol=exchange_symbol, 
                                start_date=start_date, 
                                end_date=end_date, 
                                save_path=save_path, 
                                progress_path=progress_path, 
                                min_time_between_requests=min_time_between_requests,
                                data_source_list=data_source_list,
                                return_exceptions_list=True)

Loaded 26 tickers.
Starting download of prices for 26 tickers from 2016-01-01 to 2024-06-04.
------------------------------
Completed all downloads and saved final results to raw/prices/commodities_adj_close.csv.
------------------------------
List of exceptions which occured: []


## Data Processing

- Filter out non-trading days (some stocks/etfs errorenously have prices on weekends/holidays)
  - Note: Cryptocurrencies trade continuously -> We keep two versions: one with all days and one with only US trading days (unclear at this point whether we will allow crypto trading on weekends/holidays in the portfolio simulation). 
- Add price availability information to names_and_tickers data
  - Date of first/last available price
  - Indicator for completely missing data during observation timeframe (should be mostly assets delisted before start of observation period)
    - Can be used to remove these as matching candidates! 
- Compute and save returns
    - discard assets with > x % missing price data or longest consecutive missing sequence > y (between first and last available price)
    - for the remaining assets, use forward fill to fill in missing prices before computing returns

In [2]:
start_date = "2016-01-01"
end_date = "2023-12-31" # we have ~ 5 months of data from 2024 but here we make the cutoff at the end of 2023, which is sufficient for our analysis (1 year post video scraping cutoff date).  
# get column of official US trading dates
US_trading_dates = adu.get_US_trading_dates(start_date, end_date)

# load raw price data
stocks = pd.read_csv("raw/prices/stocks_adj_close.csv", sep=";")
etfs = pd.read_csv("raw/prices/etfs_adj_close.csv", sep=";")
commodities = pd.read_csv("raw/prices/commodities_adj_close.csv", sep=";")
cryptos = pd.read_csv("raw/prices/cryptos_adj_close.csv", sep=";")

In [3]:
asset_data = [("stocks", stocks), ("etfs", etfs), ("commodities", commodities), ("cryptos", cryptos)]
#asset_data = [("commodities", commodities)] # for testing

for name, df in asset_data:
        
    print(f"{'-'*30}\nProcessing {name} data...")

    # set date col as index for the process
    df = df.set_index("date")

    ### price fixes
    # - some price series have exact zeros (either rounded from very small numbers or as placeholder for missing data, or errors)
    # - negative price values are always errors (except for 2020-04-20 in the crude oil futures contract!)
    # -> to avoid issues with returns computation, we replace these values with NaNs

    df = df.where(df > 0, other=pd.NA)

    # applying manual fixes of faulty data for specific tickers
    for fix in adu.manual_price_fixes_dict[name]:
        df.loc[fix["date"], fix["ticker"]] = fix["new_val"]
        print(f"Applied manual price fix for {fix['ticker']} on {fix['date']}.")

    ### keep only trading day rows
    prices = pd.merge(US_trading_dates.set_index("date"), df, on="date", how="left")
    ### save date-filtered prices (to prices folder, not raw/prices!)
    prices.to_csv(f"prices/{name}_adj_close.csv", sep=";", index=True, index_label="date")

    ### add data availability to names_and_tickers_with_price_availability files
    nat_df = pd.read_csv(f"names_and_tickers/{'yahoo_' if name=='commodities' else ''}eodhd_{name}.csv", sep=";")
    tcol = "Code" if name != "cryptos" else "Code_clean" # column name for tickers in nat_df
    # temporarily set ticker col as index for faster lookups
    nat_df = nat_df.set_index(tcol)
    # create new columns
    nat_df["earliest_price_date"] = None
    nat_df["latest_price_date"] = None
    nat_df["has_price_data"] = False # will be set true if any prices in observation period are available
    nat_df["n_trading_days_between_earliest_and_latest"] = None # considering only days between earliest and latest date
    nat_df["n_prices_between_earliest_and_latest"] = None
    nat_df["n_missing_prices_between_earliest_and_latest"] = None
    nat_df["missing_prices_ratio"] = None # considering only days between earliest and latest date
    nat_df["longest_missing_price_sequence"] = None # considering only days between earliest and latest date

    # compute values for priced tickers and enter them in nat_df
    # to-do if time: replace with better code, currently very inefficient
    for i, t in enumerate(prices.columns):
        
        # define shortcut references
        p = prices[t] # single price series with date index
        
        earliest_date = p.first_valid_index() # None if no data
        latest_date = p.last_valid_index() # NaN if no data
        nat_df.loc[t, "earliest_price_date"] = earliest_date
        nat_df.loc[t, "latest_price_date"] = latest_date
        has_price_data = not pd.isnull(earliest_date)
        nat_df.loc[t, "has_price_data"] = has_price_data
        if has_price_data:
            p_seq_idx = (p.index >= earliest_date) & (p.index <= latest_date)
            nat_df.loc[t, "n_trading_days_between_earliest_and_latest"] = p[p_seq_idx].shape[0]
            nat_df.loc[t, "n_prices_between_earliest_and_latest"] = p[p_seq_idx].notnull().sum()
            nat_df.loc[t, "n_missing_prices_between_earliest_and_latest"] = p[p_seq_idx].isnull().sum()
            nat_df.loc[t, "missing_prices_ratio"] = nat_df.loc[t, "n_missing_prices_between_earliest_and_latest"] / nat_df.loc[t, "n_trading_days_between_earliest_and_latest"]
            longest_missing_sequence = p[p_seq_idx].isnull().astype(int).groupby(p.notnull().cumsum()).cumcount().max()
            nat_df.loc[t, "longest_missing_price_sequence"] = longest_missing_sequence
        
        if (i+1)%100==0:
            print(f"-- {i+1}/{len(prices.columns)-1} tickers processed.")

    ### compute returns
    # drop ineligible tickers (due to too many missing prices)
    to_drop = nat_df[(~nat_df["has_price_data"]) | # no price data at all
                     (nat_df["missing_prices_ratio"] > 0.2) |# ratio of missing prices too high (we can keep this pretty high if the longest_missing_sequence is low enough. sometimes assets have a period of spotty data at the beginning or end, but also periods of good data which we want to preserve.)
                     (nat_df["longest_missing_price_sequence"] > 20) |# too many consecutive missing prices
                     (nat_df["n_prices_between_earliest_and_latest"] < 10) # too few prices in total
                     ].index.tolist()
    prices_df = prices.drop(columns=[t for t in to_drop if t in prices.columns])
    print(f"Return computation prep: Dropped {len(to_drop)} tickers due to missing data.")
    
    # get returns df
    returns_df = adu.get_returns_from_prices(prices_df, drop_full_na_cols=True)

    ### detect likely errors 
    # (e.g. excessive one-day returns due to price data errors) -> drop the affected tickers
    returns_df, dropped_errors = adu.detect_errors_in_returns(returns_df, name, drop_cols_with_likely_errors=True)
    print(f"Return computation: Dropped {len(dropped_errors)} tickers due to likely presence of errors in price data.")

    ### save returns to returns folder
    #returns_df = returns_df.reset_index() # get dates back as column
    returns_df.to_csv(f"returns/{name}_returns.csv", sep=";", index=True, index_label="date")

    ### save nat df with price info to names_and_tickers_with_price_availability folder

    # add column to indicate whether price data is likely to be faulty (-> returns should not be used, or have already been dropped)
    nat_df["price_data_errors_likely"] = nat_df.index.isin(dropped_errors)
    # add column to indicate whether returns have been computed and are available
    nat_df["has_returns"] = nat_df.index.isin(returns_df.columns)
    # reset nat index (tcol)
    nat_df = nat_df.reset_index()
    nat_df.to_csv(f"names_and_tickers_with_price_availability/{name}.csv", sep=";", index=False)

    print(f"{name} data processed and saved.")


------------------------------
Processing stocks data...
-- 100/13288 tickers processed.
-- 200/13288 tickers processed.
-- 300/13288 tickers processed.
-- 400/13288 tickers processed.
-- 500/13288 tickers processed.
-- 600/13288 tickers processed.
-- 700/13288 tickers processed.
-- 800/13288 tickers processed.
-- 900/13288 tickers processed.
-- 1000/13288 tickers processed.
-- 1100/13288 tickers processed.
-- 1200/13288 tickers processed.
-- 1300/13288 tickers processed.
-- 1400/13288 tickers processed.
-- 1500/13288 tickers processed.
-- 1600/13288 tickers processed.
-- 1700/13288 tickers processed.
-- 1800/13288 tickers processed.
-- 1900/13288 tickers processed.
-- 2000/13288 tickers processed.
-- 2100/13288 tickers processed.
-- 2200/13288 tickers processed.
-- 2300/13288 tickers processed.
-- 2400/13288 tickers processed.
-- 2500/13288 tickers processed.
-- 2600/13288 tickers processed.
-- 2700/13288 tickers processed.
-- 2800/13288 tickers processed.
-- 2900/13288 tickers proces

### Result Checks / Examination

In [4]:
# check returns dfs: are there any missing values inbetween first and last?
for name in ["stocks", "etfs", "commodities", "cryptos"]:
    returns = pd.read_csv(f"returns/{name}_returns.csv", sep=";")
    print(f"Checking {name} returns for missing sequences...")
    for i, ticker in enumerate(returns.columns[1:]):
        first_non_na = returns[ticker].first_valid_index()
        last_non_na = returns[ticker].last_valid_index()
        len_missing_sequence_between_first_and_last = returns.loc[first_non_na:last_non_na, ticker].isnull().astype(int).groupby(returns[ticker].notnull().cumsum()).cumcount().max()
        if len_missing_sequence_between_first_and_last > 0:
            print(f"{ticker}: {len_missing_sequence_between_first_and_last} missing values in sequence.")
    print(f"{name} returns checked.")

Checking stocks returns for missing sequences...
stocks returns checked.
Checking etfs returns for missing sequences...
etfs returns checked.
Checking commodities returns for missing sequences...
commodities returns checked.
Checking cryptos returns for missing sequences...
cryptos returns checked.


In [5]:
# check how many extracted & matched tickers have no returns data (at all, we don't consider video upload date here)
# if it's too many it might be worth considering to rerun the matching process and omit tickers without returns data. 
# (would be trivial with the now available has_returns column in the names_and_tickers_with_price_availability files)

import json

video_df = pd.read_csv("../matched/VIDEOS_inf_llama3_ft_v4_q8_0_llamacpp_guided.csv", sep=";")

nat_data = [("stock", pd.read_csv("names_and_tickers_with_price_availability/stocks.csv", sep=";")),
            ("etf", pd.read_csv("names_and_tickers_with_price_availability/etfs.csv", sep=";")),
            ("commodity", pd.read_csv("names_and_tickers_with_price_availability/commodities.csv", sep=";")),
            ("crypto", pd.read_csv("names_and_tickers_with_price_availability/cryptos.csv", sep=";"))
            ]
for asset_type, nat in nat_data:

    tcol = "Code" if asset_type != "crypto" else "Code_clean" # name of ticker column in nat

    # get every ticker extraction relevant for trading (= non-neutral sentiment) in a list (incl. duplicates)
    all_tickers = []
    for l in video_df["trade_info_incl_neutrals"]:
        for a in json.loads(l):
            if a["asset_type"] == asset_type and a["sentiment"] != "neutral":
                all_tickers.append(a["ticker"])
    # ticker value counts
    ticker_counts = pd.Series(all_tickers).value_counts()
    ticker_counts = pd.DataFrame(ticker_counts).reset_index().rename(columns={"index": tcol, 0: "count"})
    tickers_with_returns = nat[nat["has_returns"]][tcol].tolist()
    ticker_counts["has_return_data"] = ticker_counts[tcol].isin(tickers_with_returns)

    # print results
    print(f"{'-'*30}\n{asset_type.upper()}S:\n")
    print(f"Total extracted tickers relevant for trading: {len(all_tickers)}")
    print(f"Of those, {ticker_counts[ticker_counts['has_return_data']]['count'].sum()} have returns data.\n")
    print(f"Unique extracted tickers relevant for trading: {len(ticker_counts)}")
    print(f"Of those, {ticker_counts['has_return_data'].sum()} have returns data.\n")
    print(f"Top 50 tickers with most extractions:\n{ticker_counts.head(50)}")


------------------------------
STOCKS:

Total extracted tickers relevant for trading: 29913
Of those, 27624 have returns data.

Unique extracted tickers relevant for trading: 3525
Of those, 2623 have returns data.

Top 50 tickers with most extractions:
    Code  count  has_return_data
0   TSLA   1726             True
1   AAPL    894             True
2    NIO    754             True
3   AMZN    509             True
4   META    487             True
5    AMC    432             True
6   BABA    384             True
7   PLTR    382             True
8   MSFT    380             True
9    DIS    301             True
10  NVDA    284             True
11  DKNG    257             True
12  TTCF    219             True
13  CHPT    210             True
14   JNJ    206             True
15  GOOG    204             True
16     T    203             True
17    SQ    200             True
18   AMD    196             True
19  LCID    192             True
20    BA    188             True
21   GME    187      

In [6]:
t = "AXS-USD"
asset_type = "cryptos"
nat = pd.read_csv(f"names_and_tickers_with_price_availability/{asset_type}.csv", sep=";")
nat[nat["Code"]==t]

,Code_clean,Code,Name,Country,Exchange,Currency,Type,Isin,delisted_as_of_may_2024,in_top200_as_of_dec_2022,earliest_price_date,latest_price_date,has_price_data,n_trading_days_between_earliest_and_latest,n_prices_between_earliest_and_latest,n_missing_prices_between_earliest_and_latest,missing_prices_ratio,longest_missing_price_sequence,price_data_errors_likely,has_returns
16,AXS,AXS-USD,Axie Infinity,Unknown,CC,USD,Currency,NaN,False,True,2020-12-15,2023-12-29,True,765.0,765.0,0.0,0.0,0.0,True,False


### Benchmark Data

Separately from the other asset classes, we also want to have a returns data file with assets to be used for benchmarking or as the neutral asset in portfolio computation. The following should be included:

- cash (no downloads needed, all zero returns)
- 3 month US treasury bill (``^IRX`` on yahoo finance) as proxy for risk-free rate -> needs to be converted to daily returns! See ``get_daily_3m_tbill_returns()`` in ``asset_data_utils.py``.
- ``SPY`` as proxy for S&P 500
- ``VTI`` as proxy for US total stock market
- ``VT`` as proxy for global stock market
- ``GLD`` as proxy for gold

(etf returns are already availablee, no need to download and compute returns again)

In [12]:
start_date = "2016-01-01"
end_date = "2023-12-31"

# initialize returns df with date column
benchmark_returns = pd.DataFrame(adu.get_US_trading_dates(start_date, end_date), columns=["date"])

# get benchmark data

# 1. cash
benchmark_returns["cash"] = 0.0
# 2. download/compute new: 3 month T-bill returns
tbill_returns = adu.get_daily_3m_tbill_returns(start_date, end_date)
benchmark_returns = pd.merge(benchmark_returns, tbill_returns, on="date", how="left")
# 3. get from etf data
tickers = ["SPY", "VTI", "VT", "GLD"]
etf_returns = pd.read_csv("returns/etfs_returns.csv", sep=";")
for t in tickers:
    benchmark_returns = pd.merge(benchmark_returns, etf_returns[["date", t]], on="date", how="left")

# save benchmark returns data
benchmark_returns.to_csv("returns/benchmarks_returns.csv", sep=";", index=False)

[*********************100%%**********************]  1 of 1 completed
